In [1]:
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.8.1")

Overview: This notebook shows four variants of a tight loop and how they react to cancellation:
- No cancellation checks or suspension: cancel has no effect; loop runs to completion.
- While condition checks isActive: cooperative cancellation when requested.
- ensureActive() inside loop: throws CancellationException promptly when canceled.
- yield() inside loop: adds a suspension point, most CPU-friendly and responsive.

This first example is a tight busy loop inside runBlocking that prints "Hello i" roughly every 500 ms. It never suspends (no delay/yield), so it fully occupies the thread until it finishes. There is no cancellation check here, so once started it will run until i reaches the limit.

In [ ]:
import kotlinx.coroutines.runBlocking
import kotlinx.coroutines.Dispatchers
import kotlinx.coroutines.delay
import kotlinx.coroutines.launch

runBlocking {
    val startTime = System.currentTimeMillis()
    val job = launch(Dispatchers.Default) {
        var nextPrintTime = startTime
        var i = 0
        while (i < 5) {
            if (System.currentTimeMillis() >= nextPrintTime) {
                println("Hello ${i++}")
                nextPrintTime += 500L
            }
        }
    }
    delay(1000L)
    println("Cancel!")
    job.cancel()
    println("Done!")
}

Here we move the busy loop into a launched coroutine on Dispatchers.Default and cooperatively check isActive in the while condition. The parent delays for 1 second, prints "Cancel!", then cancels the Job. Because the loop checks isActive and still doesn’t suspend, it will stop promptly once cancellation is requested, but it can still spin CPU between prints.

In [ ]:
import kotlinx.coroutines.Dispatchers
import kotlinx.coroutines.delay
import kotlinx.coroutines.isActive
import kotlinx.coroutines.launch
import kotlinx.coroutines.runBlocking

runBlocking {
    val startTime = System.currentTimeMillis()
    val job = launch(Dispatchers.Default) {
        var nextPrintTime = startTime
        var i = 0
        while (i < 5 && isActive) {
            if (System.currentTimeMillis() >= nextPrintTime) {
                println("Hello ${i++}")
                nextPrintTime += 500L
            }
        }
    }
    delay(1000L)
    println("Cancel!")
    job.cancel()
    println("Done!")
}

This variant uses ensureActive() inside the loop. It throws CancellationException if the coroutine was canceled, immediately unwinding the coroutine. It’s still a busy loop (no suspension), but now cancellation is enforced even if you forget to check isActive in the while condition.

In [ ]:
import kotlinx.coroutines.Dispatchers
import kotlinx.coroutines.delay
import kotlinx.coroutines.ensureActive
import kotlinx.coroutines.isActive
import kotlinx.coroutines.launch
import kotlinx.coroutines.runBlocking

runBlocking {
    val startTime = System.currentTimeMillis()
    val job = launch(Dispatchers.Default) {
        var nextPrintTime = startTime
        var i = 0
        while (i < 5) {
            ensureActive()
            if (System.currentTimeMillis() >= nextPrintTime) {
                println("Hello ${i++}")
                nextPrintTime += 500L
            }
        }
    }
    delay(1000L)
    println("Cancel!")
    job.cancel()
    println("Done!")
}

Finally, yield() inserts an explicit suspension point inside the loop. Suspension gives other coroutines (including the canceller) a chance to run and promptly delivers the cancellation signal. This is the most CPU-friendly and responsive approach for loops that can cooperate.

In [15]:
import kotlinx.coroutines.Dispatchers
import kotlinx.coroutines.delay
import kotlinx.coroutines.ensureActive
import kotlinx.coroutines.isActive
import kotlinx.coroutines.launch
import kotlinx.coroutines.runBlocking
import kotlinx.coroutines.yield

runBlocking {
    val startTime = System.currentTimeMillis()
    val job = launch(Dispatchers.Default) {
        var nextPrintTime = startTime
        var i = 0
        while (i < 5) {
            yield()
            if (System.currentTimeMillis() >= nextPrintTime) {
                println("Hello ${i++}")
                nextPrintTime += 500L
            }
        }
    }
    delay(1000L)
    println("Cancel!")
    job.cancel()
    println("Done!")
}

Hello 0
Hello 1
Hello 2
Cancel!
Done!


In [14]:
import kotlinx.coroutines.Dispatchers
import kotlinx.coroutines.delay
import kotlinx.coroutines.ensureActive
import kotlinx.coroutines.isActive
import kotlinx.coroutines.launch
import kotlinx.coroutines.runBlocking
import kotlinx.coroutines.yield

runBlocking {
    val job = launch(Dispatchers.Default) {
        repeat(5) {
            println("Hello ${it}")
            delay(500L)
        }
    }
    delay(1000L)
    println("Cancel!")
    job.cancel()
    println("Done!")
}

Hello 0
Hello 1
Cancel!
Done!


In [21]:
import kotlinx.coroutines.withContext

suspend fun otherWork() {
    println("other work")
}

suspend fun someOtherWork() {
    println("someOtherWork")
}


suspend fun workOne() = withContext(Dispatchers.IO) {
    println("Working one start")
    otherWork()
    yield()
    someOtherWork()
    println("Working one end")
}

suspend fun workTwo() = withContext(Dispatchers.IO) {
    println("Working two start")
    otherWork()
    yield()
    someOtherWork()
    println("Working two end")
}

In [22]:
runBlocking {
    val job = launch {
        workOne()
        yield() // or ensureActive()
        workTwo()
    }
 }

Working one start
other work
someOtherWork
Working one end
Working two start
other work
someOtherWork
Working two end


In [23]:
runBlocking {
    val job = launch {
        workOne()
        ensureActive()
        workTwo()
    }
 }

Working one start
other work
someOtherWork
Working one end
Working two start
other work
someOtherWork
Working two end
